<a href="https://colab.research.google.com/github/pri-nitta/FIAP_IA/blob/main/LLM_com_transformers_e_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#vetor de possibilidades que entram na rede e retornam outros vetores
texto = 'Eu vou extrair os insights no banco de dados.'
caracteres = sorted(list(set(texto)))
print(caracteres)
print('Tamanho do seu vocabulário >> ', len(caracteres))

[' ', '.', 'E', 'a', 'b', 'c', 'd', 'e', 'g', 'h', 'i', 'n', 'o', 'r', 's', 't', 'u', 'v', 'x']
Tamanho do seu vocabulário >>  19


In [2]:
letraParaIndex = { lt:i for i,lt in enumerate(caracteres) }
IndexParaLetra = { i:lt for i,lt in enumerate(caracteres) }

encode = lambda s: [letraParaIndex[c] for c in s] # encoder
decode = lambda l: ''.join([IndexParaLetra[i] for i in l]) # decoder

print(encode("Eu vou extrair os insights no banco de dados."))
print(decode(encode("Eu vou extrair os insights no banco de dados.")))

[2, 16, 0, 17, 12, 16, 0, 7, 18, 15, 13, 3, 10, 13, 0, 12, 14, 0, 10, 11, 14, 10, 8, 9, 15, 14, 0, 11, 12, 0, 4, 3, 11, 5, 12, 0, 6, 7, 0, 6, 3, 6, 12, 14, 1]
Eu vou extrair os insights no banco de dados.


In [3]:
print(encode("Eu vou"))
print(decode(encode("Eu vou")))

[2, 16, 0, 17, 12, 16]
Eu vou


In [4]:
import pandas as pd
import numpy as np

texto = {'texto':[
    'Eu vou extrair os insights no banco de dados.',
    'Eu vou extrair dados no banco de dados.',
]}
texto = pd.DataFrame(texto)
texto

,texto
0,Eu vou extrair os insights no banco de dados.
1,Eu vou extrair dados no banco de dados.


In [5]:
texto["texto"].str.get_dummies(" ")

,Eu,banco,dados,dados.,de,extrair,insights,no,os,vou
0,1,1,0,1,1,1,1,1,1,1
1,1,1,1,1,1,1,0,1,0,1


In [6]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.3 MB/s eta 0:00:00


In [7]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
enc.encode('Eu vou extrair os insights no banco de dados.')

[36, 84, 25211, 3131, 343, 28686, 17218, 645, 3958, 1073, 390, 9955, 418, 13]

In [8]:
print(encode('Eu vou extrair os insights no banco de dados.'))

[2, 16, 0, 17, 12, 16, 0, 7, 18, 15, 13, 3, 10, 13, 0, 12, 14, 0, 10, 11, 14, 10, 8, 9, 15, 14, 0, 11, 12, 0, 4, 3, 11, 5, 12, 0, 6, 7, 0, 6, 3, 6, 12, 14, 1]


RAG

In [9]:
!pip install -U -q google-generativeai

In [10]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get('API_KEY')
genai.configure(api_key=api_key)

In [11]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [12]:
title = "Nesta aula você está aprendendo sobre embeddings, LLM e RAG"
sample_text = (
    "Artigo completo:\n"
    "\n"
    "A RAG permite que o LLM acesse e processe informações de bases de conhecimento externas, como artigos científicos, notícias, documentos técnicos ou bancos de dados específicos da empresa. Isso garante que as respostas do modelo sejam baseadas em fatos confiáveis e atualizados, provenientes de fontes confiáveis e relevantes para a consulta do usuário. ")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)


{'embedding': [0.045670517, -0.03429844, -0.0019491868, 0.061504688, 0.016138982, 0.012835481, 0.0069171526, -0.016790511, -0.018815242, 0.038934138, -0.003066853, 0.03520192, 0.0038993866, -0.026782006, 0.0070495713, -0.058802627, 0.001907187, 0.002491838, 0.01595095, -0.018474827, -0.017125374, 0.03210567, 0.007840017, -0.02794988, -0.0052476428, 0.013196341, 0.015133078, -0.026576204, 0.030218825, 0.03150359, -0.024429223, 0.013602308, -0.032393117, 0.02242839, 0.0177124, -0.0027792298, -0.024873707, -0.00025838427, 0.008349765, 0.011807918, 0.026914168, -0.051092014, -0.02075382, -0.004614748, 0.011940615, -0.034422614, 0.029764883, 0.034514222, -0.00833224, -0.09982131, -0.019365156, -0.0385202, 0.041938692, -0.008774487, 0.023348168, -0.023870843, 0.004962889, -0.018996645, -0.0051096017, 0.005367524, -0.0019832442, -0.021537669, -0.02380684, 0.06055746, 0.009208774, -0.035316657, -0.005873743, -0.014235458, 0.03437443, -0.019317204, -0.0053063175, -0.016284456, 0.040133145, 0.02

In [13]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.5 MB/s eta 0:00:00


In [15]:
import pandas as pd
from PyPDF2 import PdfReader

def ler_pdf(caminho_arquivo):
    """
    Lê um arquivo PDF e retorna um dicionário com título e conteúdo.

    Argumentos:
        caminho_arquivo (str): Caminho para o arquivo PDF.

    Retorna:
        dict: Dicionário com as chaves "titulo" e "conteudo".
    """

    # Abrir o arquivo PDF
    with open(caminho_arquivo, 'rb') as arquivo:
        leitor = PdfReader(arquivo)

        # Obter o título do PDF (metadados)
        titulo = leitor.metadata.get('/Title') or 'Sem título'

        # Extrair o conteúdo do PDF
        texto = ''
        for pagina in leitor.pages:
            texto += pagina.extract_text()

    # Retornar dicionário com título e conteúdo
    return {'titulo': titulo, 'conteudo': texto}

def armazenar_resultados():
    """
    Armazena os resultados extraídos dos PDFs em um DataFrame.

    Retorna:
        pd.DataFrame: DataFrame contendo os dados dos PDFs.
    """

    # Criar lista vazia para armazenar dados dos PDFs
    dados = []

    # Ler cada PDF e adicionar ao dataframe
    for arquivo in ['/content/drive/MyDrive/20 IDÉIAS PARA LANCHES.pdf']:
        dados_pdf = ler_pdf(arquivo)
        dados.append(dados_pdf)

    # Criar dataframe a partir da lista
    df = pd.DataFrame(dados)

    # Salvar dataframe (opcional)
    df.to_csv('dados_pdf.csv', index=False)

    return df

df = armazenar_resultados()

In [16]:
df.head()

,titulo,conteudo
0,20 IDÉIAS PARA LANCHES ATÉ 250 KCAL,Combinações \nPequenas\nRefeições\n20 Ideias p...


In [17]:
model = "models/embedding-001"

def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

df["Embeddings"] = df.apply(lambda row: embed_fn(row["titulo"], row["conteudo"]), axis=1)
df.head()

,titulo,conteudo,Embeddings
0,20 IDÉIAS PARA LANCHES ATÉ 250 KCAL,Combinações \nPequenas\nRefeições\n20 Ideias p...,"[-0.0037174535, 0.014662412, -0.03179558, -0.0..."


In [18]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(base["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["conteudo"]

In [21]:
consulta = "aveia"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Combinações 
Pequenas
Refeições
20 Ideias para encaixar no Café da
Manhã ou TardeD o c e s
Opção 1 
Combinações
Iogurte Desnatado 
( 160g)Aveia em Flocos
( 15g) Banana
( 75g)D o c e s
Opção 2
Combinações
Crepioca
( 1 0vo + 25 g de tapioca)Geleia de Fruta
Sugestão: Incremente sementes a crepioca :
 abóbora, linhaça, chia ou girassol(20g)D o c e s
Opção 3 
Combinações
Abacate
( 50g)Mel
( 15g)Cacau
( 30g)D o c e s
Opção 4
Aveia em FlocosChia
( 15g )
Combinações
Overnight Oats( 15g)
Como Fazer: Misture o iogurte, a chia e a aveia e
 deixe na geladeira de um dia para o outro. 
Para comer pique a fruta escolhida por cimaIogurte Desnatado
( 150g )Manga ou Fruta
( 50g )
D o c e s
Opção 5
Combinações
Salada de Frutas 
( 3 frutas)
( 200g)Semente de Chia
( 15g)D o c e s
Opção 6
Combinações
Leite em Pó Desnatado
( 30g)Morango
( 120g)Mel
( 15g)
Sugestão:Adicionar 1 colher de sopa de Chia.D o c e s
Opção 7 
Combinações
Polpa de Açaí
( 100g)Aveia em Flocos
( 20g) Banana
( 75g)
Sugestão: Bater o açaí 

In [23]:
generation_config = {
  "temperature": 0.4, # Esse parâmetro controla a aleatoriedade do texto gerado
  "candidate_count": 1, # Especifica o número de tentativas independentes de geração de texto (candidatos) a serem consideradas.
  "top_p": 0.90, # Controla a distribuição de probabilidade usada para amostrar a próxima palavra.
  "top_k": 10, # Limita o tamanho do vocabulário considerado para gerar a próxima palavra
  "max_output_tokens": 500 # Define o número máximo de tokens (palavras) que o texto gerado pode ter.
}

prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_RAG = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_RAG.generate_content(prompt)
print(response.text)


**Doces pra Alegrar o Dia**

* **Opção 1:** Iogurte com aveia, banana e um toque de doçura
* **Opção 2:** Crepioca com geleia de fruta e sementes pra dar um croc
* **Opção 3:** Abacate com mel e cacau, uma combinação cremosa e deliciosa
* **Opção 4:** Overnight oats com chia e iogurte, uma delícia que fica pronta de um dia pro outro
* **Opção 5:** Salada de frutas com chia, uma explosão de sabores e nutrientes
* **Opção 6:** Leite em pó com morango e mel, um clássico que nunca sai de moda
* **Opção 7:** Polpa de açaí com aveia e banana, um smoothie refrescante e energizante
* **Opção 8:** Sorvete proteico com whey protein e iogurte, uma sobremesa saudável e gostosa
* **Opção 9:** Bolo de banana e aveia, um quitute caseiro e nutritivo
* **Opção 10:** Smoothie de banana, iogurte e morango, uma bebida geladinha e refrescante

**Salgados pra Matar a Fome**

* **Opção 1:** Sanduíche de pão integral com pasta de atum e requeijão light
* **Opção 2:** Aveioca com frango desfiado e salada, uma 

In [24]:
response = model_RAG.generate_content(prompt)
print(response.text)

**Ideias para o Café da Manhã ou Tarde**

**Doces**

* Iogurte, aveia e banana
* Crepioca com geleia e sementes
* Abacate com mel e cacau
* Overnight oats com iogurte, chia e frutas
* Salada de frutas com chia
* Leite em pó, morango e mel
* Açaí com aveia e banana
* Sorvete proteico com whey protein e cacau
* Bolo de banana e aveia
* Smoothie de banana, iogurte e frutas

**Salgadas**

* Sanduíche de pão integral com pasta de atum
* Aveioca com frango desfiado
* Cuscuz de milho com ovos mexidos e gergelim
* Torrada integral com pasta de queijo minas
* Omelete de sardinha
* Wrap de frango desfiado ou carne moída
* Crepioca com ovo, queijo e sementes
* Crepe de espinafre com ricota ou frango
* Pão integral com banana e queijo
* Pizza de frigideira com ovo, farinha de aveia e molho de tomate


In [25]:
prompt = f"faça uma sugestão de café da manhã contendo uma opção doce e salgada: {trecho}"

response = model_RAG.generate_content(prompt)
print(response.text)

**Café da Manhã com Opções Doce e Salgada**

**Doce:**

* Crepioca com geleia de frutas e sementes de chia
* Aveia em flocos com banana, mel e cacau

**Salgada:**

* Pão integral com pasta de atum e requeijão light
* Aveioca com frango desfiado e salada
